# ⏱️ Latency and Error Analysis Notebook

This notebook analyzes **request latency** and **error latency** percentiles from Fortio JSON output files generated during HTTP load testing experiments.

## 🧪 Experiments Covered

1. **01 - HTTP Max Throughput**
    - Compares p50, p75, p90, and p99 latency and error latencies between Istio and Linkerd at max throughput.

2. **02 - HTTP Constant Throughput**
    - Runs at constant QPS values: 1, 1000, and 10000.
    - Compares latency distributions per mesh across different traffic loads.

3. **03 - HTTP Payload Variation**
    - Uses fixed QPS (100) with varying payload sizes: 0, 1000, and 10000 bytes.
    - Explores how latency changes with request payload size.

## 📁 Expected Input

- JSON files under `../results/<experiment>/latencies_<mesh>_<qps>_<payload>_<timestamp>.json`
- Structure:
    - Fortio's `DurationHistogram` and `ErrorsDurationHistogram` blocks
    - Extracted percentiles: **p50, p75, p90, p99**

## 📉 Output

- Line plots for latency and error latency percentiles (per experiment config).
- Output PNGs are saved in:
    - `../diagrams/<experiment>/experiment*_latency*.png`
    - `../diagrams/<experiment>/experiment*_error_latency*.png`

> 🧪 **Benchmark Goal**: Compare service mesh behavior under varying load and payload profiles.

In [1]:
import os
import glob
import json
import matplotlib.pyplot as plt

In [2]:
def extract_percentiles(data, error=False):
    """
    Extract percentiles (p50, p75, p90, p99) from 
    a Fortio JSON blob and convert to milliseconds.
    """
    if error:
        perc_list = data.get("ErrorsDurationHistogram", {}).get("Percentiles", [])
    else:
        perc_list = data.get("DurationHistogram", {}).get("Percentiles", [])
    result = {}
    for entry in perc_list:
        p = entry.get("Percentile")
        if p in [50, 75, 90, 99]:
            # Multiply by 1000 to convert seconds to milliseconds.
            result[p] = entry.get("Value") * 1000 if entry.get("Value") is not None else None
    return result

def extract_actual_qps(data):
    """
    Extracts the actual QPS from a Fortio JSON blob,
    rounded to the nearest integer (no decimal part).
    """
    raw = data.get("ActualQPS")
    if raw is None:
        return None
    return int(round(raw))

def extract_requested_qps(data):
    """
    Extracts the requested QPS from a Fortio JSON blob.
    """
    return data.get("RequestedQPS", None)

def extract_params_from_filename(file_path):
    """
    Parses a filename of the form:
       latencies_<mesh>_<qps>_<payload>_<timestamp>.json
    and returns (mesh, qps, payload).
    """
    base = os.path.basename(file_path)
    base = base[len("latencies_"):]  # Remove prefix.
    base = base.replace(".json", "")
    parts = base.split("_")
    mesh = parts[0]
    qps = int(parts[1])
    payload = int(parts[2])
    return mesh, qps, payload

def get_mesh_color(mesh):
    """
    Returns the plot color for the given mesh name.
    Defaults to 'black' if the mesh is not in the color_map.
    """
    color_map = {
        'istio': 'orange',
        'linkerd': 'blue',
        'baseline': 'green'
    }
    return color_map.get(mesh.lower(), 'black')

In [3]:
# HTTP Max throughput experiment (experiment 1).
experiment_pattern = os.path.join("..", "results", "*", "01_http_max_throughput")
diagram_dir        = os.path.join("..", "diagrams", "01_http_max_throughput")
latency_files      = glob.glob(os.path.join(experiment_pattern, "latencies_*.json"))
os.makedirs(diagram_dir, exist_ok=True)

# Dictionaries to store results per mesh.
raw_latency    = {}  
raw_error      = {}    
raw_actual_qps = {}
requested_qps  = {}

for file in latency_files:
    with open(file, 'r') as f:
        data = json.load(f)
    mesh, qps, payload = extract_params_from_filename(file)
    raw_latency    .setdefault(mesh, []).append( extract_percentiles(data, error=False) )
    raw_error      .setdefault(mesh, []).append( extract_percentiles(data, error=True)  )
    raw_actual_qps .setdefault(mesh, []).append( extract_actual_qps(data) )
    requested_qps = extract_requested_qps(data)

# Compute averages
results_latency = {
    mesh: { p: sum(d.get(p,0) for d in digs)/len(digs) for p in (50,75,90,99)} 
    for mesh, digs in raw_latency.items()
}
results_error = {
    mesh: { p: sum(d.get(p,0) for d in digs)/len(digs) for p in (50,75,90,99)}
    for mesh, digs in raw_error.items()
}
actual_qps_per_mesh = {
    mesh: sum(vals)/len(vals)
    for mesh, vals in raw_actual_qps.items()
}

# Actual QPS per mesh
plt.figure(figsize=(8,6))
meshes = list(actual_qps_per_mesh.keys())
actuals = [actual_qps_per_mesh[m] for m in meshes]
plt.bar(meshes, actuals)
plt.xlabel('Service Mesh')
plt.ylabel('Actual QPS')
plt.title(f"Experiment 1: Actual QPS per Mesh\n(Prot=HTTP, Requested QPS={requested_qps})")
plt.tight_layout()
actual_qps_path = os.path.join(diagram_dir, 'actual_qps.png')
plt.savefig(actual_qps_path)
plt.close()

# Define x-axis labels.
x_labels = ['p50', 'p75', 'p90', 'p99']

# Plot latency percentiles.
for mesh, percs in results_latency.items():
    y_values = [percs.get(50), percs.get(75), percs.get(90), percs.get(99)]
    color = get_mesh_color(mesh)
    plt.figure(figsize=(8,6))
    plt.plot(x_labels, y_values, color=color, marker='o')
    plt.xlabel('Percentile')
    plt.ylabel('Latency (ms)')
    plt.title(f'Experiment 1: Latency Percentiles for {mesh}\n(Prot=HTTP, Actual QPS={actual_qps_per_mesh[mesh]}, Requested QPS={requested_qps})')
    plt.tight_layout()
    out_path = os.path.join(diagram_dir, f'latency_{mesh}.png')
    plt.savefig(out_path)
    plt.close()

# Error Latency Plot
for mesh, percs in results_error.items():
    y_values = [percs.get(50), percs.get(75), percs.get(90), percs.get(99)]
    color = get_mesh_color(mesh)
    plt.figure(figsize=(8,6))
    plt.plot(x_labels, y_values, color=color, marker='o')
    plt.xlabel('Percentile')
    plt.ylabel('Error Latency (ms)')
    plt.title(f'Experiment 1: Error Latency Percentiles for {mesh}\n(Prot=HTTP, Actual QPS={actual_qps_per_mesh[mesh]}, Requested QPS={requested_qps})')
    plt.tight_layout()
    out_path = os.path.join(diagram_dir, f'error_latency_{mesh}.png')
    plt.savefig(out_path)
    plt.close()
    

In [4]:
# gRPC Max throughput experiment (experiment 2).
experiment_pattern = os.path.join("..", "results", "*", "02_grpc_max_throughput")
diagram_dir        = os.path.join("..", "diagrams", "02_grpc_max_throughput")
latency_files      = glob.glob(os.path.join(experiment_pattern, "latencies_*.json"))
os.makedirs(diagram_dir, exist_ok=True)

# Dictionaries to store results per mesh.
raw_latency     = {}   
raw_error       = {}
raw_actual_qps  = {}  
requested_qps   = None

for file in latency_files:
    with open(file, 'r') as f:
        data = json.load(f)
    mesh, qps, payload = extract_params_from_filename(file)
    raw_latency    .setdefault(mesh, []).append(extract_percentiles(data, error=False))
    raw_error      .setdefault(mesh, []).append(extract_percentiles(data, error=True))
    raw_actual_qps .setdefault(mesh, []).append(extract_actual_qps(data))
    requested_qps = extract_requested_qps(data)

# Compute averages
results_latency     = {
    mesh: { p: sum(d.get(p,0) for d in digs)/len(digs) for p in (50,75,90,99) }
    for mesh, digs in raw_latency.items()
}
results_error       = {
    mesh: { p: sum(d.get(p,0) for d in digs)/len(digs) for p in (50,75,90,99) }
    for mesh, digs in raw_error.items()
}
actual_qps_per_mesh = {
    mesh: sum(vals)/len(vals)
    for mesh, vals in raw_actual_qps.items()
}

# Actual QPS per mesh
plt.figure(figsize=(8,6))
meshes = list(actual_qps_per_mesh.keys())
actuals = [actual_qps_per_mesh[m] for m in meshes]
plt.bar(meshes, actuals)
plt.xlabel('Service Mesh')
plt.ylabel('Actual QPS')
plt.title(f'Experiment 2: Actual QPS per Mesh\n(Prot=gRPC, Requested QPS={requested_qps})')
plt.tight_layout()
actual_qps_path = os.path.join(diagram_dir, 'actual_qps.png')
plt.savefig(actual_qps_path)
plt.close()

# Define x-axis labels.
x_labels = ['p50', 'p75', 'p90', 'p99']

# Plot latency percentiles.
for mesh, percs in results_latency.items():
    y_values = [percs.get(50), percs.get(75), percs.get(90), percs.get(99)]
    color = get_mesh_color(mesh)
    plt.figure(figsize=(8,6))
    plt.plot(x_labels, y_values, color=color, marker='o')
    plt.xlabel('Percentile')
    plt.ylabel('Latency (ms)')
    plt.title(f'Experiment 2: Latency Percentiles for {mesh}\n(Prot=gRPC, Actual QPS={actual_qps_per_mesh[mesh]}, Requested QPS={requested_qps})')
    plt.tight_layout()
    out_path = os.path.join(diagram_dir, f'latency_{mesh}.png')
    plt.savefig(out_path)
    plt.close()

# Error Latency Plot
for mesh, percs in results_error.items():
    y_values = [percs.get(50), percs.get(75), percs.get(90), percs.get(99)]
    color = get_mesh_color(mesh)
    plt.figure(figsize=(8,6))
    plt.plot(x_labels, y_values, color=color, marker='o')
    plt.xlabel('Percentile')
    plt.ylabel('Error Latency (ms)')
    plt.title(f'Experiment 2: Error Latency Percentiles for {mesh}\n(Prot=gRPC, Actual QPS={actual_qps_per_mesh[mesh]}, Requested QPS={requested_qps})')
    plt.tight_layout()
    out_path = os.path.join(diagram_dir, f'error_latency_{mesh}.png')
    plt.savefig(out_path)
    plt.close()

In [5]:
# HTTP Constant throughput experiment (experiment 3).
experiment_pattern = os.path.join("..", "results", "*", "03_http_constant_throughput")
diagram_dir        = os.path.join("..", "diagrams", "03_http_constant_throughput")
latency_files      = glob.glob(os.path.join(experiment_pattern, "latencies_*.json"))
os.makedirs(diagram_dir, exist_ok=True)

# Dictionaries to store results per mesh.
raw_latency    = {} 
raw_error      = {}
raw_actual_qps = {}

for file in latency_files:
    with open(file, 'r') as f:
        data = json.load(f)
    mesh, qps, payload = extract_params_from_filename(file)
    if qps not in results_latency:
        results_latency[qps] = {}
        results_error[qps] = {}
        actual_qps_per_mesh[qps] = {}
    raw_latency   .setdefault(qps, {}).setdefault(mesh, []).append(extract_percentiles(data, error=False))
    raw_error     .setdefault(qps, {}).setdefault(mesh, []).append(extract_percentiles(data, error=True))
    raw_actual_qps.setdefault(qps, {}).setdefault(mesh, []).append(extract_actual_qps(data))

# compute averages
results_latency = {
    qps: {
        mesh: { p: sum(d.get(p,0) for d in digs)/len(digs) for p in (50,75,90,99) }
        for mesh, digs in mesh_dict.items()
    }
    for qps, mesh_dict in raw_latency.items()
}
results_error = {
    qps: {
        mesh: { p: sum(d.get(p,0) for d in digs)/len(digs) for p in (50,75,90,99) }
        for mesh, digs in mesh_dict.items()
    }
    for qps, mesh_dict in raw_error.items()
}
actual_qps_per_mesh = {
    qps: {
        mesh: sum(vals)/len(vals)
        for mesh, vals in mesh_vals.items()
    }
    for qps, mesh_vals in raw_actual_qps.items()
}

# Define x-axis labels.
x_labels = ["p50", "p75", "p90", "p99"]

# Plot latency percentiles.
for qps_val in sorted(results_latency.keys()):
    plt.figure(figsize=(15, 10))
    for mesh, percs in results_latency[qps_val].items():
        actual = actual_qps_per_mesh[qps_val][mesh]
        color = get_mesh_color(mesh)
        y_values = [percs.get(50), percs.get(75), percs.get(90), percs.get(99)]
        plt.plot(x_labels, y_values, marker='o', color=color, label=f"{mesh} (Actual QPS={actual})")
    plt.xlabel("Percentile")
    plt.ylabel("Latency (ms)")
    plt.title(f"Experiment 3: Latency Percentiles\n(Prot=HTTP, Requested QPS={qps_val})")
    plt.legend()
    plt.tight_layout()
    latency_outfile = os.path.join(diagram_dir, f"latency_{qps_val}_0.png")
    plt.savefig(latency_outfile)
    plt.close()

    # Error Latency Plot
    plt.figure(figsize=(15, 10))
    for mesh, percs in results_error[qps_val].items():
        actual = actual_qps_per_mesh[qps_val][mesh] 
        color = get_mesh_color(mesh)
        y_values = [percs.get(50), percs.get(75), percs.get(90), percs.get(99)]
        plt.plot(x_labels, y_values, marker='o', color=color, label=f"{mesh} (Actual QPS={actual})")
    plt.xlabel("Percentile")
    plt.ylabel("Error Latency (ms)")
    plt.title(f"Experiment 3: Latency Errors Percentiles\n(Prot=HTTP, Requested QPS={qps_val})")
    plt.legend()
    plt.tight_layout()
    error_outfile = os.path.join(diagram_dir, f"latency_error_{qps_val}_0.png")
    plt.savefig(error_outfile)
    plt.close()

In [6]:
# gRPC Constant throughput experiment (experiment 4).
experiment_pattern = os.path.join("..", "results", "*", "04_grpc_constant_throughput")
diagram_dir        = os.path.join("..", "diagrams", "04_grpc_constant_throughput")
latency_files      = glob.glob(os.path.join(experiment_pattern, "latencies_*.json"))
os.makedirs(diagram_dir, exist_ok=True)

# Dictionaries to store results per mesh.
raw_latency    = {} 
raw_error      = {}
raw_actual_qps = {}

for file in latency_files:
    with open(file, 'r') as f:
        data = json.load(f)
    mesh, qps, payload = extract_params_from_filename(file)
    if qps not in results_latency:
        results_latency[qps] = {}
        results_error[qps] = {}
        actual_qps_per_mesh[qps] = {}
    raw_latency   .setdefault(qps, {}).setdefault(mesh, []).append(extract_percentiles(data, error=False))
    raw_error     .setdefault(qps, {}).setdefault(mesh, []).append(extract_percentiles(data, error=True))
    raw_actual_qps.setdefault(qps, {}).setdefault(mesh, []).append(extract_actual_qps(data))

# compute averages
results_latency = {
    qps: {
        mesh: { p: sum(d.get(p,0) for d in digs)/len(digs) for p in (50,75,90,99) }
        for mesh, digs in mesh_dict.items()
    }
    for qps, mesh_dict in raw_latency.items()
}
results_error = {
    qps: {
        mesh: { p: sum(d.get(p,0) for d in digs)/len(digs) for p in (50,75,90,99) }
        for mesh, digs in mesh_dict.items()
    }
    for qps, mesh_dict in raw_error.items()
}
actual_qps_per_mesh = {
    qps: {
        mesh: sum(vals)/len(vals)
        for mesh, vals in mesh_vals.items()
    }
    for qps, mesh_vals in raw_actual_qps.items()
}

# Define x-axis labels.
x_labels = ["p50", "p75", "p90", "p99"]

# Plot latency percentiles.
for qps_val in sorted(results_latency.keys()):
    plt.figure(figsize=(15, 10))
    for mesh, percs in results_latency[qps_val].items():
        actual = actual_qps_per_mesh[qps_val][mesh]
        color = get_mesh_color(mesh)
        y_values = [percs.get(50), percs.get(75), percs.get(90), percs.get(99)]
        plt.plot(x_labels, y_values, marker='o', color=color, label=f"{mesh} (Actual QPS={actual})")
    plt.xlabel("Percentile")
    plt.ylabel("Latency (ms)")
    plt.title(f"Experiment 4: Latency Percentiles\n(Prot=gRPC, Requested QPS={qps_val})")
    plt.legend()
    plt.tight_layout()
    latency_outfile = os.path.join(diagram_dir, f"latency_{qps_val}_0.png")
    plt.savefig(latency_outfile)
    plt.close()

    # Error Latency Plot
    plt.figure(figsize=(15, 10))
    for mesh, percs in results_error[qps_val].items():
        actual = actual_qps_per_mesh[qps_val][mesh] 
        color = get_mesh_color(mesh)
        y_values = [percs.get(50), percs.get(75), percs.get(90), percs.get(99)]
        plt.plot(x_labels, y_values, marker='o', color=color, label=f"{mesh} (Actual QPS={actual})")
    plt.xlabel("Percentile")
    plt.ylabel("Error Latency (ms)")
    plt.title(f"Experiment 4: Latency Errors Percentiles\n(Prot=gRPC, Requested QPS={qps_val})")
    plt.legend()
    plt.tight_layout()
    error_outfile = os.path.join(diagram_dir, f"latency_error_{qps_val}_0.png")
    plt.savefig(error_outfile)
    plt.close()

In [7]:
# HTTP Constant throughput with Payload experiment (experiment 5).
experiment_pattern = os.path.join("..", "results", "*", "05_http_payload")
diagram_dir        = os.path.join("..", "diagrams", "05_http_payload")
latency_files      = glob.glob(os.path.join(experiment_pattern, "latencies_*.json"))
os.makedirs(diagram_dir, exist_ok=True)

# Define x-axis labels.
raw_latency    = {}
raw_error      = {}
raw_actual_qps = {}
requested_qps  = None

for file in latency_files:
    with open(file, 'r') as f:
        data = json.load(f)
    mesh, qps, payload = extract_params_from_filename(file)
    if payload not in results_latency:
        results_latency[payload] = {}
        results_error[payload] = {}
        actual_qps_per_mesh[payload] = {}
    raw_latency    .setdefault(payload, {}).setdefault(mesh, []).append(extract_percentiles(data, error=False))
    raw_error      .setdefault(payload, {}).setdefault(mesh, []).append(extract_percentiles(data, error=True))
    raw_actual_qps .setdefault(payload, {}).setdefault(mesh, []).append(extract_actual_qps(data))
    requested_qps = extract_requested_qps(data)

# compute averages
results_latency = {
    payload: {
        mesh: { p: sum(d.get(p,0) for d in digs)/len(digs) for p in (50,75,90,99) }
        for mesh, digs in mesh_dict.items()
    }
    for payload, mesh_dict in raw_latency.items()
}
results_error = {
    payload: {
        mesh: { p: sum(d.get(p,0) for d in digs)/len(digs) for p in (50,75,90,99) }
        for mesh, digs in mesh_dict.items()
    }
    for payload, mesh_dict in raw_error.items()
}
actual_qps_per_mesh = {
    payload: {
        mesh: sum(vals)/len(vals)
        for mesh, vals in mesh_vals.items()
    }
    for payload, mesh_vals in raw_actual_qps.items()
}

# Define x-axis labels.
x_labels = ["p50", "p75", "p90", "p99"]

# Plot latency percentiles.
for payload_val in sorted(results_latency.keys()):
    plt.figure(figsize=(15, 10))
    for mesh, percs in results_latency[payload_val].items():
        actual = actual_qps_per_mesh[payload_val][mesh] 
        color = get_mesh_color(mesh)
        y_values = [percs.get(50), percs.get(75), percs.get(90), percs.get(99)]
        plt.plot(x_labels, y_values, marker='o', color=color, label=f"{mesh} (Actual QPS={actual})")
    plt.xlabel("Percentile")
    plt.ylabel("Latency (ms)")
    plt.title(f"Experiment 5: Latency Percentiles\n(Prot=HTTP, Requested QPS={requested_qps}, Payload={payload_val})")
    plt.legend()
    plt.tight_layout()
    latency_outfile = os.path.join(diagram_dir, f"latency_{requested_qps}_{payload_val}.png")
    plt.savefig(latency_outfile)
    plt.close()
    
    # Error Latency Plot
    plt.figure(figsize=(15, 10))
    for mesh, percs in results_error[payload_val].items():
        actual = actual_qps_per_mesh[payload_val][mesh]
        color = get_mesh_color(mesh)
        y_values = [percs.get(50), percs.get(75), percs.get(90), percs.get(99)]
        plt.plot(x_labels, y_values, marker='o', color=color, label=f"{mesh} (Actual QPS={actual})")
    plt.xlabel("Percentile")
    plt.ylabel("Error Latency (ms)")
    plt.title(f"Experiment 5: Latency Errors Percentiles\n(Prot=HTTP, Requested QPS={requested_qps}, Payload={payload_val})")
    plt.legend()
    plt.tight_layout()
    error_outfile = os.path.join(diagram_dir, f"latency_error_{requested_qps}_{payload_val}.png")
    plt.savefig(error_outfile)
    plt.close()

In [8]:
# HTTP Constant throughput with HTTPRoute header-based routing experiment (experiment 6).
experiment_pattern = os.path.join("..", "results", "*", "06_http_constant_throughput_header")
diagram_dir        = os.path.join("..", "diagrams", "06_http_constant_throughput_header")
latency_files      = glob.glob(os.path.join(experiment_pattern, "latencies_*.json"))
os.makedirs(diagram_dir, exist_ok=True)

# Dictionaries to store results per mesh.
raw_latency    = {}
raw_error      = {}
raw_actual_qps = {}

for file in latency_files:
    with open(file, 'r') as f:
        data = json.load(f)
    mesh, qps, payload = extract_params_from_filename(file)
    if qps not in results_latency:
        results_latency[qps] = {}
        results_error[qps] = {}
        actual_qps_per_mesh[qps] = {}
    raw_latency   .setdefault(qps, {}).setdefault(mesh, []).append(extract_percentiles(data, error=False))
    raw_error     .setdefault(qps, {}).setdefault(mesh, []).append(extract_percentiles(data, error=True))
    raw_actual_qps.setdefault(qps, {}).setdefault(mesh, []).append(extract_actual_qps(data))

# compute averages
results_latency = {
    qps: {
        mesh: { p: sum(d.get(p,0) for d in digs)/len(digs) for p in (50,75,90,99) }
        for mesh, digs in mesh_dict.items()
    }
    for qps, mesh_dict in raw_latency.items()
}
results_error = {
    qps: {
        mesh: { p: sum(d.get(p,0) for d in digs)/len(digs) for p in (50,75,90,99) }
        for mesh, digs in mesh_dict.items()
    }
    for qps, mesh_dict in raw_error.items()
}
actual_qps_per_mesh = {
    qps: {
        mesh: sum(vals)/len(vals)
        for mesh, vals in mesh_vals.items()
    }
    for qps, mesh_vals in raw_actual_qps.items()
}

# Define x-axis labels.
x_labels = ["p50", "p75", "p90", "p99"]

# Plot latency percentiles.
for qps_val in sorted(results_latency.keys()):
    plt.figure(figsize=(15, 10))
    for mesh, percs in results_latency[qps_val].items():
        actual = actual_qps_per_mesh[qps_val][mesh]
        color = get_mesh_color(mesh)
        y_values = [percs.get(50), percs.get(75), percs.get(90), percs.get(99)]
        plt.plot(x_labels, y_values, marker='o', color=color, label=f"{mesh} (Actual QPS={actual})")
    plt.xlabel("Percentile")
    plt.ylabel("Latency (ms)")
    plt.title(f"Experiment 6: Latency Percentiles\n(Prot=HTTP, Requested QPS={qps_val})")
    plt.legend()
    plt.tight_layout()
    latency_outfile = os.path.join(diagram_dir, f"latency_{qps_val}_0.png")
    plt.savefig(latency_outfile)
    plt.close()

    # Error Latency Plot
    plt.figure(figsize=(15, 10))
    for mesh, percs in results_error[qps_val].items():
        actual = actual_qps_per_mesh[qps_val][mesh]
        color = get_mesh_color(mesh)
        y_values = [percs.get(50), percs.get(75), percs.get(90), percs.get(99)]
        color = get_mesh_color(mesh)
        plt.plot(x_labels, y_values, marker='o', color=color, label=f"{mesh} (Actual QPS={actual})")
    plt.xlabel("Percentile")
    plt.ylabel("Error Latency (ms)")
    plt.title(f"Experiment 6: Latency Error Percentiles\n(Prot=HTTP, Requested QPS={qps_val})")
    plt.legend()
    plt.tight_layout()
    error_outfile = os.path.join(diagram_dir, f"latency_error_{qps_val}_0.png")
    plt.savefig(error_outfile)
    plt.close()